### Загрузка данных

In [2]:
!pip install pymorphy2
!pip install gradio

In [3]:
!pip install natasha
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from natasha import Segmenter
import pickle
import re
import string
import nltk
import pymorphy2
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gradio as gr

In [79]:
dff = pd.read_csv('/home/user/Загрузки/train_dataset_Минстрой(1)/df1.csv', index_col=0)
dff.head()

,id,name(название),responsibilities(Должностные обязанности),requirements(Требования к соискателю),terms(Условия),responsibilities
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,***,***,Работа на строительных площадках. Очистных соо...
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,***,***,Гнутье арматурной стали на механических станка...
2,9495851,Арматурщик,Вязка арматуры.,***,***,Вязка арматуры.
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,***,.Выполнение работ по сборке и вязке арматурны...
4,9388857,Арматурщик,Вахта в город .Москва. Обязанности: армировани...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Вахта в город .Москва. Обязанности: армировани...


In [80]:
dff.rename(columns = {'name(название)': 'Название', 'responsibilities(Должностные обязанности)':'Описание', 'requirements(Требования к соискателю)':'Требования', 'terms(Условия)': 'Условия', 'responsibilities': 'Обязанности'}, inplace = True )

In [81]:
dff.head()

,id,Название,Описание,Требования,Условия,Обязанности
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,***,***,Работа на строительных площадках. Очистных соо...
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,***,***,Гнутье арматурной стали на механических станка...
2,9495851,Арматурщик,Вязка арматуры.,***,***,Вязка арматуры.
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,Выполнение работ по гнутью и резке арматурной ...,***,.Выполнение работ по сборке и вязке арматурны...
4,9388857,Арматурщик,Вахта в город .Москва. Обязанности: армировани...,опыт в строительстве приветствуется; работа в ...,продолжительность вахты 60/30 (продление вахты...,Вахта в город .Москва. Обязанности: армировани...


### Обработка данных для загрузки в модель

In [82]:
df_new = dff.copy()

In [83]:
# Разбиваем на предложения
segmenter = Segmenter()
def sentenize_text(text):
  sentences = [s.text for s in segmenter.sentenize(text)]
  return sentences

In [84]:
# Создание тренировочного датасета с метками классов
def split_to_sent(df, col_list):
    df_train = pd.DataFrame()
    for i, col in enumerate(col_list):
        df_splited = pd.DataFrame()
        df_splited['param'] = df[col].apply(sentenize_text)
        df_splited = df_splited.explode('param')
        df_splited['class'] = col
        print(len(df_splited))
        df_train = pd.concat([df_train, df_splited], ignore_index=False)
        df_train = df_train.loc[df_train['param'] != '***']
    return df_train

In [85]:
df_train = split_to_sent(df_new, df_new.columns[3:6].tolist())

1559
2857
3503


In [86]:
# Смотрим дисбаланс классов
df_train['class'].value_counts()

Обязанности    3500
Условия        2760
Требования     1388
Name: class, dtype: int64

In [87]:
# Удаление дубликатов строк
df_train.drop_duplicates(subset='param', inplace=True)
df_train['class'].value_counts()

Обязанности    2214
Условия        1867
Требования     1051
Name: class, dtype: int64

In [88]:
df_train.head()

,param,class
3,Выполнение работ по гнутью и резке арматурной ...,Требования
4,опыт в строительстве приветствуется; работа в ...,Требования
5,"Опыт работы от 1 года , наличие удостоверения ...",Требования
6,опыт в строительстве приветствуется -работа в ...,Требования
11,Понимание работы в бригаде · .Дисциплинированн...,Требования


In [89]:
df_train.reset_index(inplace=True, drop=True)
df_train.head()

,param,class
0,Выполнение работ по гнутью и резке арматурной ...,Требования
1,опыт в строительстве приветствуется; работа в ...,Требования
2,"Опыт работы от 1 года , наличие удостоверения ...",Требования
3,опыт в строительстве приветствуется -работа в ...,Требования
4,Понимание работы в бригаде · .Дисциплинированн...,Требования


In [90]:
# Сохранение датасета
df_train.to_parquet('/home/user/Загрузки/train_dataset_Минстрой(1)/df_train.parquet.gzip', compression='gzip')

In [91]:
raw_data = df_train.copy()

### Убираем стоп слова и делаем лемматизацию

In [92]:
spec_chars = string.punctuation + '\n\xa0«»\t—…' # пунктуация
stop_words = stopwords.words('russian') # стоп-слова
morph = pymorphy2.MorphAnalyzer() # лемматизатор
def clean_text(text):
    text = re.sub(r'\s+', ' ', text, flags=re.I)  # удаление лишних пробелов
    text = text.lower()  # перевод в нижний регистр
    text = "".join([ch for ch in text if ch not in spec_chars]) #удаление пунктуации
    words = nltk.word_tokenize(text) # токенизация

    # удаление стоп-слов и лемматизация
    cleaned_words = [morph.parse(w)[0].normal_form for w in words if not w in stop_words and len(w)>1]
    return ' '.join(cleaned_words)

raw_data['clean_text'] = raw_data.param.apply(clean_text)

### Делим на тренировочные и тестовые данные

In [93]:
from sklearn.model_selection import train_test_split
X = raw_data ['clean_text']
y = raw_data ['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state= 42, test_size=0.2)

### Подготавливаем и обучаем модель на данных

In [94]:
from catboost import CatBoostClassifier, Pool
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(task_type='GPU', iterations = 10000,
                               random_seed=43,
                               leaf_estimation_method='Newton',
                               loss_function='MultiClass',
                               use_best_model=True,
                               eval_metric='TotalF1',
                               od_type='Iter',
                               od_wait=500,
                               **kwargs)

    return model.fit(train_pool, eval_set=test_pool,
                     verbose=100, plot=True,
                     use_best_model=True)

In [95]:
train_pool = Pool(data=X_train.to_frame(), label=y_train, text_features=['clean_text',], feature_names=['clean_text'])
valid_pool = Pool(data=X_test.to_frame(), label=y_test, text_features=['clean_text',], feature_names=['clean_text'])

In [96]:
model = fit_model(train_pool, valid_pool, learning_rate=0.1, depth=12, bagging_temperature=0.7, random_strength=0.7,
                  dictionaries = [{
                      'dictionary_id':'Word',
                      'max_dictionary_size': '50000'
                  }],
                 feature_calcers = ['BoW:top_tokens_count=10000'])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6735543	test: 0.6646235	best: 0.6646235 (0)	total: 121ms	remaining: 20m 6s
100:	learn: 0.8032948	test: 0.7689309	best: 0.7708161 (96)	total: 8.84s	remaining: 14m 26s
200:	learn: 0.8298805	test: 0.7727611	best: 0.7729642 (143)	total: 16.8s	remaining: 13m 40s
300:	learn: 0.8470377	test: 0.7766982	best: 0.7775004 (249)	total: 24.5s	remaining: 13m 10s
400:	learn: 0.8593367	test: 0.7811780	best: 0.7814043 (333)	total: 32.3s	remaining: 12m 53s
500:	learn: 0.8701314	test: 0.7828734	best: 0.7830680 (464)	total: 40.1s	remaining: 12m 41s
600:	learn: 0.8802262	test: 0.7846434	best: 0.7867225 (509)	total: 48.1s	remaining: 12m 32s
700:	learn: 0.8882418	test: 0.7895283	best: 0.7895283 (700)	total: 56s	remaining: 12m 22s
800:	learn: 0.8955407	test: 0.7875742	best: 0.7895283 (700)	total: 1m 3s	remaining: 12m 13s
900:	learn: 0.9036298	test: 0.7855461	best: 0.7912740 (883)	total: 1m 11s	remaining: 12m 5s
1000:	learn: 0.9107652	test: 0.7844363	best: 0.7912740 (883)	total: 1m 19s	remaining: 11

In [ ]:
model = fit_model(train_pool, valid_pool, learning_rate=0.1, depth=12, bagging_temperature=0.7, random_strength=0.7,
                  dictionaries = [{
                      'dictionary_id':'Word',
                      'max_dictionary_size': '50000'
                  }],
                 feature_calcers = ['BoW:top_tokens_count=10000'])

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7085034	test: 0.7028262	best: 0.7028262 (0)	total: 151ms	remaining: 25m 9s
100:	learn: 0.8872939	test: 0.8838090	best: 0.8838090 (100)	total: 8.89s	remaining: 14m 31s
200:	learn: 0.9063642	test: 0.9007602	best: 0.9007602 (200)	total: 17.1s	remaining: 13m 54s
300:	learn: 0.9158892	test: 0.9102773	best: 0.9102773 (300)	total: 25.2s	remaining: 13m 31s
400:	learn: 0.9206701	test: 0.9134416	best: 0.9134416 (397)	total: 33.3s	remaining: 13m 18s
500:	learn: 0.9238251	test: 0.9152720	best: 0.9153718 (485)	total: 41.6s	remaining: 13m 8s
600:	learn: 0.9266442	test: 0.9169952	best: 0.9170941 (598)	total: 48.7s	remaining: 12m 40s
700:	learn: 0.9285106	test: 0.9181433	best: 0.9181443 (692)	total: 59.5s	remaining: 13m 8s
800:	learn: 0.9303489	test: 0.9198695	best: 0.9198695 (798)	total: 1m 7s	remaining: 12m 59s
900:	learn: 0.9317102	test: 0.9213077	best: 0.9213077 (893)	total: 1m 14s	remaining: 12m 37s
1000:	learn: 0.9331208	test: 0.9220766	best: 0.9220766 (996)	total: 1m 23s	remaining: 

### Предсказания модели на валидационнвых данных и подсчет метрик

In [97]:
y_pred = model.predict(valid_pool)

In [98]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 Обязанности       0.83      0.78      0.80       440
  Требования       0.84      0.70      0.77       228
     Условия       0.74      0.86      0.79       359

    accuracy                           0.79      1027
   macro avg       0.80      0.78      0.79      1027
weighted avg       0.80      0.79      0.79      1027



In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 Обязанности       0.91      0.95      0.93      4482
  Требования       0.95      0.91      0.93      3441
     Условия       0.94      0.93      0.93      2552

    accuracy                           0.93     10475
   macro avg       0.94      0.93      0.93     10475
weighted avg       0.93      0.93      0.93     10475



### Предсказания модели на всем датасете

In [ ]:
# df = pd.read_parquet('/content/df')

In [99]:
# Разбиваем колонку "Описание" на предложения
df_test = split_to_sent(df_new, [df_new.columns[2]])

6281


In [100]:
df_test['clean_text'] = raw_data.param.apply(clean_text)
df_test['index'] = df_test.index

In [101]:
df_test.head()

,param,class,clean_text,index
0,Работа на строительных площадках.,Описание,выполнение работа гнутьё резка арматурный стат...,0
0,Очистных сооружениях.,Описание,выполнение работа гнутьё резка арматурный стат...,0
1,Гнутье арматурной стали на механических станка...,Описание,опыт строительство приветствоваться работа бри...,1
1,Сборка и установка сеток и плоских...,Описание,опыт строительство приветствоваться работа бри...,1
2,Вязка арматуры.,Описание,опыт работа год наличие удостоверение документ...,2


In [102]:
# Выполняем предсказания
df_test['pred'] = df_test['clean_text'].apply(lambda x: ''.join(model.predict([x])))
df_test['index'] = df_test['index']

In [103]:
df_test.to_parquet('/home/user/Загрузки/train_dataset_Минстрой(1)/raw_data', compression='gzip')

In [104]:
# Функция для вывода вакансии
def make_final_file(df):
  duties = '\n'.join(df.loc[df['pred'] == 'Обязанности', 'param'].values.tolist())
  requirements = '\n'.join(df.loc[df['pred'] == 'Требования', 'param'].values.tolist())
  conditions = '\n'.join(df.loc[df['pred'] == 'Условия', 'param'].values.tolist())
  final_file = f'Обязанности:\n\n{duties}\n\nТребования:\n\n{requirements}\n\nУсловия\n\n{conditions}'
  return final_file

In [105]:
some_vacancy = df_test.loc[df_test['index'] == 5]

In [ ]:
print(make_final_file(some_vacancy))

Обязанности:

Требуется АРМАТУРЩИК на крупнейший завод по производству железобетонных конструкций 3-4 разряда!
Обязанности: Вязка арматуры для заливки конструкции железобетонных изделий.

Требования:

Требования: Опыт работы от 1 года , наличие удостоверения (документы должны присутствовать + запись в трудовой книжке).
Желание работать и зарабатывать Условия: Работа вахтой от 45/60/90/120 смен;

Условия

Ставка за смену 3080 руб при графике 6/1 по 11 часов.
Предоставляем час обеда + перерывы 10-15 мин.
Трудоустройство и заселение в общежитие в день обращения;
Бесплатное проживание в комфортном общежитии (постельное бельё предоставляем);
Питание 1 раз в день предоставляется на заводе В общежитии есть кухня и все необходимое для приготовления пищи;
Работа в закрытом теплом цехе Удобный график работы ночь или день;
-Вахта от 45 смен, зп на руки 138 600.
Иногородним предоставляется проживание бесплатное в общежитии Объект расположен по адресу : гор. Подпорожье Ленинградская область.
Рассма

### Сохранение и загрузка модели

In [106]:
filename = '/home/user/Загрузки/train_dataset_Минстрой(1)/finalized_model.pickle'
# pickle.dump(model, open(filename, 'wb'))

In [107]:
with open(filename, 'rb') as f:
   model = pickle.load(f)

### Создание итоговой таблицы

In [108]:
# Группируем в итоговую таблицу
def final_table(df):  # помещаем df c предсказаниями модели
  vacancy_table = df.groupby(['index', 'pred'],as_index=False)['param'].agg(lambda x: ' '.join(x))
  tab = pd.DataFrame()
  final_table = pd.DataFrame()
  for i in vacancy_table['index'].unique():
    for col in vacancy_table['pred'].unique():
      try:
        tab[col] = vacancy_table.loc[(vacancy_table['pred'] == col) & (vacancy_table['index'] == int(i)) , 'param'].values
      except:
        tab[''] = ''
    tab['index'] = int(i)
    final_table = pd.concat([final_table, tab], ignore_index=True)
  return df

In [109]:
final_table = final_table(df_test)

In [110]:
concat_df = df_new[['id', 'Название']]

In [111]:
stacked = pd.merge(concat_df, final_table, left_index=True, right_index=True)

In [112]:
stacked.drop('index', axis=1, inplace=True)

In [ ]:
stacked

,id,Название,Обязанности,,Требования,Условия
0,9495846,Арматурщик,Работа на строительных площадках. Очистных соо...,,NaN,NaN
1,9495850,Арматурщик,Гнутье арматурной стали на механических станка...,,NaN,NaN
2,9495851,Арматурщик,Вязка арматуры.,,NaN,NaN
3,2,Арматурщик,Выполнение работ по гнутью и резке арматурной ...,,NaN,NaN
4,9388857,Арматурщик,Обязанности: армирование каркаса; Питание трех...,,Требования: опыт в строительстве приветствуетс...,Вахта в город Москва. Условия: продолжительнос...
...,...,...,...,...,...,...
994,9306889,Инженер,Обязанности: Проектирование ультразвуковых пре...,,ООО &quot;Ньювент-Уфа&quot; стаж работы желате...,"Требуются соискатели граждане РФ, с паспортом ..."
995,9306890,Монтажник,"Требуется монтажник вентиляции, возможно учени...",,Приветствуется опыт по установке систем кондиц...,"Внимание пока обьявление висит, ведется набор!..."
996,9306891,Слесарь-Ремонтник,Требуется слесарь в цех по ремонту и балансиро...,,Токарные навыки и сварочные навыки приветствую...,"Зарплата, график работы обсуждаются."
997,9306892,Арматурщик,К нам в команду требуются: . Мастер по оклейке...,,Мастер по оклейке автомобилей в цветные пленки...,"Оплата высокая, сдельная, выплаты по факту вып..."


In [ ]:
# Сохранение в эксель
stacked.to_excel('/home/user/Загрузки/train_dataset_Минстрой(1)/output.xlsx', index=False)

### Для наглядности используем gradio

Структурирование вакансии

In [113]:
# Разбивка текста на предложения
def split_to_sent_text(df):
    df_splited = pd.DataFrame()
    df_splited['text'] = df['text'].apply(lambda x: x.split('\n'))
    df_splited = df_splited.explode('text')
    df_splited = df_splited.replace(r'^\s*$', np.nan, regex=True)
    df_splited.dropna(inplace=True)
    return df_splited

In [114]:
# Преобразование текста в датафрейм
def text_to_df(text):
  df = pd.DataFrame()
  df['text'] = [text]
  return df

In [115]:
def make_final_vacancy(text):
  df = text_to_df(text)
  df_splited = split_to_sent_text(df)
  print('ok')
  df_splited['clean_text'] = df_splited['text'].apply(clean_text)
  df_splited['pred'] = df_splited['clean_text'].apply(lambda x: ''.join(model.predict([x])))
  duties = '\n'.join(df_splited.loc[df_splited['pred'] == 'Обязанности', 'text'].values.tolist())
  requirements = '\n'.join(df_splited.loc[df_splited['pred'] == 'Требования', 'text'].values.tolist())
  conditions = '\n'.join(df_splited.loc[df_splited['pred'] == 'Условия', 'text'].values.tolist())
  final_file = f'Обязанности:\n\n{duties}\n\nТребования:\n\n{requirements}\n\nУсловия\n\n{conditions}'
  return final_file

In [116]:
text = '''Планирование, организация и контроль работы подрядных организаций, выполняющих ремонтные работы;
Планирование, организация ремонтных работ;
Заключение договоров подряда, контроль выполнения;
Контроль корректности документов на выполненные работы и полученные материалы (КС-2, КС-3, акты, накладные, счета-фактуры);
Подготовка и контроль исполнения бюджета работ;
Взаимодействие с управляющими органами ТЦ по вопросам строительных работ;
Контроль соблюдения правил безопасности на объектах во время выполнения строительных работ;
Передача закрывающей и исполнительной документации после завершения работ в службы Заказчика.


Высшее/среднее профессиональное техническое образование;
Релевантный опыт работы от 2-х лет в крупных розничных сетях;
Координация и контроль работ по выполнению поставленных задач;
Опыт работы с подрядными организациями, проведение переговоров;
Уверенные знания по составу, наполнению и методики чтения проектной документации (стадий РД, ИД);
Уверенные знания в области инженерии, строительных работ, СНиП, ПТЭЭП, ПУЭ, ПБ, ЭБ, ОТ;
Готовность к командировкам.


Работу в комфортабельном и уютном офисе в БЦ Кунцево Плаза в 5 минутах пешком от м. Молодежная;
График работы 5/2, 9.30-18.30;
Оформление в соответствии с ТК РФ, официальную заработную плату (выплата 2 раза в месяц), оплачиваемый больничный лист, оплачиваемый отпуск 28 календарных дней;
ДМС включая стоматологию, корпоративного врача (условия в зависимости от стажа работы);
Дисконтную карту на продукцию компании, участие в распродажах в офисе;
Корпоративные скидки на фитнес;
Реальные перспективы профессионального и карьерного роста;
А еще уютные лаундж-зоны, технологичные переговорные, просторные кухни и вкусный капучино.'''

In [ ]:
# create the Gradio interface
iface = gr.Interface(fn=make_final_vacancy, inputs=gr.inputs.Textbox(label="Text"), outputs="text")

# launch the Gradio interface
iface.launch()

In [ ]:
def filter_rows(str1, str2):
    df = pd.read_excel('/home/user/Загрузки/train_dataset_Минстрой(1)/output.xlsx')
    mask = df["Условия"].str.lower().str.contains(str1) | df["Требования"].str.lower().str.contains(str1) | df["Условия"].str.lower().str.contains(str2) | df["Требования"].str.lower().str.contains(str2)
    result = df[mask]
    #result[["id", "Name"]]
    #return result.to_string(index=False)
    #return result[["id", "name"]]
    return result[['id', "Название", 'Обязанности']].to_string(index=False, header=False)
#     return result.columns[result.columns == "name"]

iface = gr.Interface(fn=filter_rows,
                     inputs=[gr.inputs.Textbox(label="Я хочу"),
                             gr.inputs.Textbox(label="У меня имеется")],
                     outputs=gr.outputs.Textbox(label="Результат"))

iface.launch()